# Utils

In [30]:
from itertools import product
from typing import Tuple, Callable

import numpy as np
import numpy.linalg as linalg
import pandas as pd
import plotly.graph_objects as go

In [31]:
def calculate_spectral_condition_number(matrix: np.ndarray) -> float:
    return linalg.cond(matrix)


def calculate_volume_condition_number(matrix: np.ndarray) -> float:
    volume_0 = abs(linalg.det(matrix))  # Объём косоугольного параллелепипеда

    volume = 1  # Объём прямоугольного параллелепипеда
    for row in matrix:
        volume *= linalg.norm(row)

    return volume / volume_0


def calculate_angle_condition_number(matrix: np.ndarray) -> float:
    inverse_matrix = linalg.inv(matrix)

    candidates = []
    for row, column in zip(matrix, inverse_matrix.T):
        candidates.append(linalg.norm(row) * linalg.norm(column))

    return max(candidates)

In [32]:
def banchmark_linear_system(matrix: np.ndarray, right_part: np.ndarray, bias: float) -> pd.Series:
    true_solution = linalg.solve(matrix, right_part)

    spectral_condition_number = calculate_spectral_condition_number(matrix)
    volume_condition_number = calculate_volume_condition_number(matrix)
    angle_condition_number = calculate_angle_condition_number(matrix)

    perturbed_matrix = matrix - bias
    perturbed_right_part = right_part - bias

    perturbed_matrix_solution = linalg.solve(perturbed_matrix, right_part)
    perturbed_right_part_solution = linalg.solve(matrix, perturbed_right_part)
    perturbed_linear_system_solution = linalg.solve(perturbed_matrix, perturbed_right_part)

    return pd.Series(
        [
            spectral_condition_number,
            volume_condition_number,
            angle_condition_number,
            linalg.norm(perturbed_matrix_solution - true_solution),
            linalg.norm(perturbed_right_part_solution - true_solution),
            linalg.norm(perturbed_linear_system_solution - true_solution),
        ]
    )

In [33]:
def compare_size_and_cond(data: pd.DataFrame) -> None:
    fig = go.Figure()

    data = data[['n', 'cond_s', 'cond_v', 'cond_a']].drop_duplicates()

    fig.add_scatter(
        x=data['n'], 
        y=data['cond_s'],
        name='Спектральный критерий',
    )

    fig.add_scatter(
        x=data['n'], 
        y=data['cond_v'],
        name='Объёмный критерий',
    )

    fig.add_scatter(
        x=data['n'], 
        y=data['cond_a'],
        name='Угловой критерий',
    )

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Число обусловленности', tickformat='.2e')
    fig.update_layout(title='Зависимость числа обусловленности от размера матрицы')

    fig.show()

In [34]:
def compare_cond_and_error(data: pd.DataFrame, bias: float = 1e-6, error_name: str = 'error_matrix') -> None:
    fig = go.Figure()

    fig.add_scatter(
        x=data[data['bias'] == bias]['cond_s'],
        y=data[data['bias'] == bias][error_name],
        name='Спектральный критерий',
        mode='markers',
    )

    fig.add_scatter(
        x=data[data['bias'] == bias]['cond_v'],
        y=data[data['bias'] == bias][error_name],
        name='Объёмный критерий',
        mode='markers',
    )

    fig.add_scatter(
        x=data[data['bias'] == bias]['cond_a'],
        y=data[data['bias'] == bias][error_name],
        name='Угловой критерий',
        mode='markers',
    )

    fig.update_xaxes(title='Число обусловленности', tickformat='.2e')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title=f'Зависимость ошибки от числа обусловленности ({bias = :.0e}, {error_name = })')

    fig.show()

In [35]:
def compare_size_and_error(data: pd.DataFrame, *, bias: float = 1e-6) -> None:
    fig = go.Figure()

    fig.add_scatter(
        x=data[data['bias'] == bias]['n'],
        y=data[data['bias'] == bias]['error_matrix'],
        name='Матрица',
    )

    fig.add_scatter(
        x=data[data['bias'] == bias]['n'],
        y=data[data['bias'] == bias]['error_right'],
        name='Правая часть',
    )

    fig.add_scatter(
        x=data[data['bias'] == bias]['n'],
        y=data[data['bias'] == bias]['error_system'],
        name='Вся система',
    )

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title=f'Зависимость ошибки от размера матрицы ({bias = :.0e})')

    fig.show()

In [36]:
def compare_bias_and_error(data: pd.DataFrame, n: int) -> None:
    fig = go.Figure()

    fig.add_scatter(
        x=data[data['n'] == n]['bias'],
        y=data[data['n'] == n]['error_matrix'],
        name='Матрица',
    )

    fig.add_scatter(
        x=data[data['n'] == n]['bias'],
        y=data[data['n'] == n]['error_right'],
        name='Правая часть',
    )

    fig.add_scatter(
        x=data[data['n'] == n]['bias'],
        y=data[data['n'] == n]['error_system'],
        name='Вся система',
    )

    fig.update_xaxes(title='bias', type='log', tickformat='.0e')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title=f'Зависимость ошибки от bias ({n = })')

    fig.show()

In [37]:
def generate_matrix(element_factory: Callable[[int, int], float], size: int) -> np.ndarray:
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ]
    )

# Diagonal (Bad for spectral)

In [38]:
def get_spectral_bad_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray]:
    matrix = np.eye(size)
    matrix[0][0] = 10 ** size

    right_part = np.dot(matrix, np.ones(size))

    return matrix, right_part

In [39]:
spectral_bad_data = pd.DataFrame(product(range(2, 11), [10 ** -i for i in range(2, 11)]), columns=['n', 'bias'])
spectral_bad_data[['cond_s', 'cond_v', 'cond_a', 'error_matrix', 'error_right', 'error_system']] = spectral_bad_data.apply(
    func=lambda row: banchmark_linear_system(*get_spectral_bad_linear_system(int(row.n)), row.bias),
    axis=1,
)
spectral_bad_data

,n,bias,cond_s,cond_v,cond_a,error_matrix,error_right,error_system
0,2,1.000000e-02,1.000000e+02,1.0,1.0,2.020507e-02,1.000050e-02,1.010254e-02
1,2,1.000000e-03,1.000000e+02,1.0,1.0,2.002122e-03,1.000050e-03,1.001061e-03
2,2,1.000000e-04,1.000000e+02,1.0,1.0,2.000302e-04,1.000050e-04,1.000151e-04
3,2,1.000000e-05,1.000000e+02,1.0,1.0,2.000120e-05,1.000050e-05,1.000060e-05
4,2,1.000000e-06,1.000000e+02,1.0,1.0,2.000102e-06,1.000050e-06,1.000051e-06
...,...,...,...,...,...,...,...,...
76,10,1.000000e-06,1.000000e+10,1.0,1.0,3.000027e-05,3.000000e-06,2.700024e-05
77,10,1.000000e-07,1.000000e+10,1.0,1.0,3.000003e-06,3.000000e-07,2.700002e-06
78,10,1.000000e-08,1.000000e+10,1.0,1.0,3.000000e-07,3.000000e-08,2.700000e-07
79,10,1.000000e-09,1.000000e+10,1.0,1.0,3.000000e-08,3.000000e-09,2.700000e-08


In [40]:
compare_size_and_cond(spectral_bad_data)

Unsupported

In [41]:
compare_cond_and_error(spectral_bad_data, bias=1e-2)
compare_cond_and_error(spectral_bad_data, bias=1e-6)
compare_cond_and_error(spectral_bad_data, bias=1e-10)

Unsupported

In [42]:
compare_size_and_error(spectral_bad_data, bias=1e-2)
compare_size_and_error(spectral_bad_data, bias=1e-6)
compare_size_and_error(spectral_bad_data, bias=1e-10)

Unsupported

In [43]:
compare_bias_and_error(spectral_bad_data, 2)
compare_bias_and_error(spectral_bad_data, 6)
compare_bias_and_error(spectral_bad_data, 10)

Unsupported

# Diagonally dominant (Bad for volume)

In [44]:
def get_volume_bad_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray]:
    matrix = np.eye(size) * 2 + np.eye(size, k=-1) * -1 + np.eye(size, k=1) * -1
    
    right_part = np.dot(matrix, np.ones(size))

    return matrix, right_part

In [45]:
volume_bad_data = pd.DataFrame(product(range(2, 11), [10 ** -i for i in range(2, 11)]), columns=['n', 'bias'])
volume_bad_data[['cond_s', 'cond_v', 'cond_a', 'error_matrix', 'error_right', 'error_system']] = volume_bad_data.apply(
    func=lambda row: banchmark_linear_system(*get_volume_bad_linear_system(int(row.n)), row.bias),
    axis=1,
)
volume_bad_data

,n,bias,cond_s,cond_v,cond_a,error_matrix,error_right,error_system
0,2,1.000000e-02,3.00000,1.666667,1.666667,2.886150e-02,1.414214e-02,1.443075e-02
1,2,1.000000e-03,3.00000,1.666667,1.666667,2.834095e-03,1.414214e-03,1.417048e-03
2,2,1.000000e-04,3.00000,1.666667,1.666667,2.828993e-04,1.414214e-04,1.414496e-04
3,2,1.000000e-05,3.00000,1.666667,1.666667,2.828484e-05,1.414214e-05,1.414242e-05
4,2,1.000000e-06,3.00000,1.666667,1.666667,2.828433e-06,1.414214e-06,1.414216e-06
...,...,...,...,...,...,...,...,...
76,10,1.000000e-06,48.37415,589.090909,12.898203,3.663735e-04,3.663332e-05,3.297361e-04
77,10,1.000000e-07,48.37415,589.090909,12.898203,3.663372e-05,3.663332e-06,3.297035e-05
78,10,1.000000e-08,48.37415,589.090909,12.898203,3.663336e-06,3.663332e-07,3.297002e-06
79,10,1.000000e-09,48.37415,589.090909,12.898203,3.663332e-07,3.663332e-08,3.296999e-07


In [46]:
compare_size_and_cond(volume_bad_data)

Unsupported

In [47]:
compare_cond_and_error(volume_bad_data, bias=1e-2)
compare_cond_and_error(volume_bad_data, bias=1e-6)
compare_cond_and_error(volume_bad_data, bias=1e-10)

Unsupported

In [48]:
compare_size_and_error(volume_bad_data, bias=1e-2)
compare_size_and_error(volume_bad_data, bias=1e-6)
compare_size_and_error(volume_bad_data, bias=1e-10)

Unsupported

In [49]:
compare_bias_and_error(volume_bad_data, n=2)
compare_bias_and_error(volume_bad_data, n=6)
compare_bias_and_error(volume_bad_data, n=10)

Unsupported

# Hilbert

In [50]:
def get_hilbert_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray]:
    matrix = np.around(generate_matrix(lambda row, column: 1 / (row + column -1), size), 10)
    
    right_part = np.dot(matrix, np.ones(size))

    return matrix, right_part

In [51]:
hilbert_data = pd.DataFrame(product(range(2, 11), [10 ** -i for i in range(2, 11)]), columns=['n', 'bias'])
hilbert_data[['cond_s', 'cond_v', 'cond_a', 'error_matrix', 'error_right', 'error_system']] = hilbert_data.apply(
    func=lambda row: banchmark_linear_system(*get_hilbert_linear_system(int(row.n)), row.bias),
    axis=1,
)
hilbert_data

,n,bias,cond_s,cond_v,cond_a,error_matrix,error_right,error_system
0,2,1.000000e-02,1.928147e+01,8.062258e+00,8.062258e+00,0.131762,0.063246,0.065881
1,2,1.000000e-03,1.928147e+01,8.062258e+00,8.062258e+00,0.012700,0.006325,0.006350
2,2,1.000000e-04,1.928147e+01,8.062258e+00,8.062258e+00,0.001265,0.000632,0.000633
3,2,1.000000e-05,1.928147e+01,8.062258e+00,8.062258e+00,0.000126,0.000063,0.000063
4,2,1.000000e-06,1.928147e+01,8.062258e+00,8.062258e+00,0.000013,0.000006,0.000006
...,...,...,...,...,...,...,...,...
76,10,1.000000e-06,1.029944e+11,2.224547e+46,1.250090e+10,11.603248,1.160268,10.442918
77,10,1.000000e-07,1.029944e+11,2.224547e+46,1.250090e+10,1.160272,0.116020,1.044246
78,10,1.000000e-08,1.029944e+11,2.224547e+46,1.250090e+10,0.116033,0.011603,0.104425
79,10,1.000000e-09,1.029944e+11,2.224547e+46,1.250090e+10,0.011609,0.001159,0.010450


In [52]:
compare_size_and_cond(hilbert_data)

Unsupported

In [53]:
compare_cond_and_error(hilbert_data, bias=1e-2)
compare_cond_and_error(hilbert_data, bias=1e-6)
compare_cond_and_error(hilbert_data, bias=1e-10)

Unsupported

In [54]:
compare_size_and_error(hilbert_data, bias=1e-2)
compare_size_and_error(hilbert_data, bias=1e-6)
compare_size_and_error(hilbert_data, bias=1e-10)

Unsupported

In [55]:
compare_bias_and_error(hilbert_data, n=2)
compare_bias_and_error(hilbert_data, n=5)
compare_bias_and_error(hilbert_data, n=10)

Unsupported

# Good

In [56]:
def get_good_linear_system() -> Tuple[np.ndarray, np.ndarray]:
    matrix = np.array([[-400.6, 199.8], [1198.8, -600.4]])
    right_part = np.dot(matrix, np.ones(2))
    return matrix, right_part

In [57]:
good_data = pd.DataFrame(product([2], [10 ** -i for i in range(2, 11)]), columns=['n', 'bias'])
good_data[['cond_s', 'cond_v', 'cond_a', 'error_matrix', 'error_right', 'error_system']] = good_data.apply(
    func=lambda row: banchmark_linear_system(*get_good_linear_system(), row.bias),
    axis=1,
)
good_data

,n,bias,cond_s,cond_v,cond_a,error_matrix,error_right,error_system
0,2,1.000000e-02,1998.0015,600.200033,600.200033,3.492997e-02,1.788407e-02,1.746498e-02
1,2,1.000000e-03,1998.0015,600.200033,600.200033,3.568252e-03,1.788407e-03,1.784126e-03
2,2,1.000000e-04,1998.0015,600.200033,600.200033,3.575956e-04,1.788407e-04,1.787978e-04
3,2,1.000000e-05,1998.0015,600.200033,600.200033,3.576729e-05,1.788407e-05,1.788364e-05
4,2,1.000000e-06,1998.0015,600.200033,600.200033,3.576806e-06,1.788407e-06,1.788403e-06
5,2,1.000000e-07,1998.0015,600.200033,600.200033,3.576814e-07,1.788407e-07,1.788408e-07
6,2,1.000000e-08,1998.0015,600.200033,600.200033,3.576818e-08,1.788409e-08,1.788409e-08
7,2,1.000000e-09,1998.0015,600.200033,600.200033,3.576864e-09,1.788375e-09,1.788451e-09
8,2,1.000000e-10,1998.0015,600.200033,600.200033,3.577433e-10,1.788339e-10,1.788717e-10


In [58]:
compare_bias_and_error(good_data, n=2)

Unsupported